<a href="https://colab.research.google.com/github/Map987/download-anime-mag-/blob/main/failed%20original%20keyvisual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Map987/download-anime-mag-.git

Cloning into 'download-anime-mag-'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 108 (delta 24), reused 0 (delta 0), pack-reused 61
Receiving objects: 100% (108/108), 2.72 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os

folders = ["/content/sample_data/outputtxt", "/content/sample_data/outtxt", "/content/sample_data/imagebig/"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

In [4]:
import os
import re
import xml.etree.ElementTree as ET
from typing import List

def extract_links(xml_string: str) -> List[str]:
    root = ET.fromstring(xml_string)
    links = []
    for url in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
        image_loc = url.find('{http://www.google.com/schemas/sitemap-image/1.1}image').find('{http://www.google.com/schemas/sitemap-image/1.1}loc').text
        if image_loc:
            links.append(image_loc)
    return links

In [5]:
def write_links_to_file(file_path: str, links: List[str]) -> None:
    with open(file_path, 'w') as f:
        for link in links:
            f.write(link)

In [ ]:
input_folder = '/content/download-anime-mag-/'
output_folder = '/content/sample_data/outputtxt/'

for file_name in os.listdir(input_folder):
    if file_name.endswith('.xml'):
        input_path = os.path.join(input_folder, file_name)
        with open(input_path, 'r') as f:
            xml_string = f.read()
        links = extract_links(xml_string)
        output_path = os.path.join(output_folder, file_name[:-4] + '.txt')
        write_links_to_file(output_path, links)
        print(f'{output_path} written successfully.\n')

In [9]:
import os

input_folder_path = '/content/sample_data/outputtxt/'
output_folder_path = '/content/sample_data/outtxt/'

# get all .txt files in input folder
files = [f for f in os.listdir(input_folder_path) if os.path.isfile(os.path.join(input_folder_path, f)) and f.endswith('.txt')]

for file_name in files:
    # open input file
    input_file_path = os.path.join(input_folder_path, file_name)
    with open(input_file_path, 'r') as input_file:
        file_content = input_file.read()

    # add newline before each https
    file_content = file_content.replace('https://', 'https://')

    # create output file
    output_file_path = os.path.join(output_folder_path, file_name)
    with open(output_file_path, 'w') as output_file:
        output_file.write(file_content)

In [10]:
import os

outtxt_dir = "/content/sample_data/outtxt"
data_dir = "/content/sample_data/"

# 遍历outtxt_dir目录中的所有txt文件
for filename in os.listdir(outtxt_dir):
    if filename.endswith(".txt") and filename.startswith("image-sitemap-"):
        # 获取X值
        x = filename.split("-")[-1].split(".")[0]

        # 创建imageX文件夹
        folder = os.path.join(data_dir, f"image{x}")
        os.makedirs(folder, exist_ok=True)

In [25]:
for x in range(1, 60):
      with open(f'/content/sample_data/outtxt/image-sitemap-{x}.txt') as f:
          links = f.read()

          link_list = links.split('https://')

          count = 0
          with open(f'/content/sample_data/out/image-sitemap-{x}.txt', 'w') as outf:
              for link in link_list:
                  if '-e' in link:
                      index = link.index('-e')
                      link = link[:index] + link[link.index('.', index):]
                      count += 1
                      outf.write(f'https://{link}\n')

In [ ]:

 import requests
 import os
 import zipfile
 import shutil
 from concurrent.futures import ThreadPoolExecutor, as_completed

 out_dir = '/content/sample_data/out'
 img_dir = '/content/sample_data/out/imageof'

 os.makedirs(img_dir, exist_ok=True)

 def download_image(url):
     img_name = url.split('/')[-1]
     r = requests.get(url, allow_redirects=True)
     with open(os.path.join(img_dir, img_name), 'wb') as f:
         f.write(r.content)

 with ThreadPoolExecutor(max_workers=16) as executor:
     for x in range(1, 61):
         with open(f'{out_dir}/image-sitemap-{x}.txt') as f:
             link_list = f.read().split()
             img_urls = [url for url in link_list if url.startswith('https://')]

         futures = [executor.submit(download_image, url) for url in img_urls]
         for future in as_completed(futures):
             future.result()

In [ ]:

 import os
 import zipfile

 out_dir = '/content/sample_data/out'
 img_dir = '/content/sample_data/out/imageof'
 zip_file = '/content/sample_data/imageof.zip'

 # Zip the imageof directory
 with zipfile.ZipFile(zip_file, 'w') as zip_f:
     for root, dirs, files in os.walk(img_dir):
         for file in files:
             zip_f.write(os.path.join(root, file))

 # Get zip file size
 stat_info = os.stat(zip_file)
 size_mb = stat_info.st_size / (1024 * 1024)
 print(f'Zip file size: {size_mb:.2f} MB')

Zip file size: 18539.19 MB


In [ ]:

 import os
 import zipfile

 out_dir = '/content/sample_data/out'
 img_dir = '/content/sample_data/out/imageof'
 zip_file = '/content/sample_data/imageof.zip'

 # Get total size of imageof folder
 total_size = 0
 for root, dirs, files in os.walk(img_dir):
     for file in files:
         total_size += os.path.getsize(os.path.join(root, file))

 # Split into 5 equal parts
 part_size = total_size // 5

 # Zip the imageof directory in parts
 for i in range(5):
     zip_file_part = zip_file.replace('.zip', f'_part{i}.zip')
     with zipfile.ZipFile(zip_file_part, 'w') as zip_f:
         size_so_far = 0
         for root, dirs, files in os.walk(img_dir):
             for file in files:
                 filename = os.path.join(root, file)
                 file_size = os.path.getsize(filename)
                 if size_so_far + file_size < part_size:
                     zip_f.write(filename)
                     size_so_far += file_size
                 else:
                     break

     # Get zip file part size
     stat_info = os.stat(zip_file_part)
     size_mb = stat_info.st_size / (1024 * 1024)
     print(f'Part {i} size: {size_mb:.2f} MB')

Part 0 size: 3706.67 MB
Part 1 size: 3706.67 MB
Part 2 size: 3706.67 MB
Part 3 size: 3706.67 MB
Part 4 size: 3706.67 MB


In [ ]:

 import os
 import shutil

 out_dir = '/content/sample_data'
 drive_dir = '/content/drive/MyDrive'
 zip_file = '/content/sample_data/imageof_part0.zip'

 # Move zip file to drive directory
 shutil.move(zip_file, drive_dir)

'/content/drive/MyDrive/imageof_part0.zip'